In [3]:
from library_import import *
from config_path import PATH
warnings.simplefilter("ignore")


In [16]:
def zip_content():
    # Importer le chemin de configuration depuis un module externe
    from config_path import PATH

    # Ouvrir le fichier ZIP en mode lecture
    parquet_files = []

    with zipfile.ZipFile(f"{PATH}input/parquet_origine.zip", 'r') as zip_ref:
        for file_info in zip_ref.infolist():
            if file_info.filename.endswith('.parquet'):
                # Extraire uniquement le nom du fichier sans le chemin
                file_name = os.path.basename(file_info.filename)
                parquet_files.append(file_name.split('.')[0] )

        # Extraire la dernière année des données en utilisant le dernier élément de la série
        last_data_year = f'20{parquet_files.iloc[-1][-2:]}'

    # Retourner la série et la dernière année des données
    return [parquet_files, last_data_year]
dataset_list, last_data_year = zip_content()

AttributeError: 'list' object has no attribute 'iloc'

In [7]:
def get_individual_source(source, rentree):
    import pandas as pd, zipfile
    from step1_data_init.sise_content import src_load

    filename = f'{source}{str(rentree)[2:4]}'
    print(filename)

    from config_path import PATH
    with zipfile.ZipFile(f"{PATH}input/parquet_origine.zip", 'r') as z:
        df = pd.read_parquet(z.open(f'{filename}.parquet'), engine='pyarrow')
    return df

In [8]:
df=get_individual_source('mana', '2013')

mana13


In [11]:
df.TYPREPA.unique()

array([None, '12202', '11205', '12201', '13201', '12203', '11101',
       '13202', 'A', '11201', '12205', '11203', '1', '13101', '12206',
       '11207', '12204', '13203', '11210', '12207', '12102', '11211',
       '11206', '11209', '11213', '11202', 'D', '12101', '11103', '11109',
       '2A EC', '2A  E', '1A LE', '1A EC', 'MP (M', '2A EN', '2A GR',
       'PC (P', '1A EN', 'PC* (', 'MPSI', 'ENS C', '2A SA', '12103',
       '13103', '12104', '11215', '12105', '?', '0', '11204', '11105',
       '11212', '13205', '11110'], dtype=object)

In [2]:
global CONF
CONF=json.load(open('utils/config_sise.json', 'r'))

In [4]:
def correctif_vars(df):
    for conf in CONF:
        var_sise = conf["var_sise"]
        format_type = conf["format"]
        missing_value = conf["missing_value"]

        if var_sise in df.columns:
            if format_type=='str':
                df[var_sise] = df[var_sise].astype(format_type)
                df[var_sise] = df[var_sise].str.split('.0', regex=False).str[0].str.strip()
                df.loc[df[var_sise].str.contains(r'^(nan|none)$', regex=True, case=False), var_sise] = ''        
                df = df.mask(df=='')
                df[var_sise] = df[var_sise].fillna(missing_value)

            if format_type=='int':
                df[var_sise] = pd.to_numeric(df[var_sise], errors='coerce').astype('Int64')

    return df


def src_load(filename, source, rentree):
    from config_path import PATH
    from utils.constants import get_vars
    with zipfile.ZipFile(f"{PATH}input/parquet_origine.zip", 'r') as z:
        df = pd.read_parquet(z.open(f'parquet_origine/{filename}.parquet'), engine='pyarrow')

    # list columns and lowercase name, create vars RENTREE/SOURCE
    df_vars = df[df.columns[df.columns.str.lower().isin([conf.get('var_sise') for conf in CONF])]]
    df_vars.columns = df_vars.columns.str.lower()
    df_vars = df_vars.assign(rentree=rentree, source=source)
    return df_vars

In [2]:
# ALL_RENTREES = list(range(2004, 2004+1))
ALL_RENTREES = [2013]
for rentree in ALL_RENTREES:
    df_all = pd.DataFrame()
    # sources = get_sources(rentree)
    sources = ['mana']
    for source in sources:

        filename = f'{source}{str(rentree)[2:4]}'
        print(filename)


        # chargement des tables en conservant que les variables de la liste utils/vars_list
        df = src_load(filename, source, rentree)

        # to check the data from the new datasets
        # with pd.ExcelWriter(excel_path, mode='a', if_sheet_exists="replace") as writer:  
        #     pd.DataFrame({"name": df.columns, "non-nulls": len(df)-df.isnull().sum().values, "nulls": df.isnull().sum().values}).to_excel(writer, sheet_name=filename, index=False)

        # df_all = pd.concat([df_all, df], ignore_index=True)
    # df_all = correctif_vars(df_all)


mana13


In [4]:
df.typrepa.unique()

array([None, '12202', '11205', '12201', '13201', '12203', '11101',
       '13202', 'A', '11201', '12205', '11203', '1', '13101', '12206',
       '11207', '12204', '13203', '11210', '12207', '12102', '11211',
       '11206', '11209', '11213', '11202', 'D', '12101', '11103', '11109',
       '2A EC', '2A  E', '1A LE', '1A EC', 'MP (M', '2A EN', '2A GR',
       'PC (P', '1A EN', 'PC* (', 'MPSI', 'ENS C', '2A SA', '12103',
       '13103', '12104', '11215', '12105', '?', '0', '11204', '11105',
       '11212', '13205', '11110'], dtype=object)

In [21]:
# df_all.head(5)
df_all.value_counts(['rentree', 'source'])

rentree  source  
2017     inscri      1953678
         culture       78298
         enq26bis      26625
Name: count, dtype: int64

In [22]:
df_all.net.unique()

<IntegerArray>
[0, 1, <NA>]
Length: 3, dtype: Int64

In [23]:
# df.info()
df_all.head(5)
# df.voie.unique()

,compos,inspr,regime,annais,sexe,bac,anbac,situpre,conv,dipder,...,effectif,groupe,cursus_lmd,voie,flag_meef,etabli_diffusion,rentree,source,oppos,par_type
0,0671864M,O,10,1993,1,S,2014,D,-1,D,...,1,4,L,0,0,UNIVERSITE STRASBOURG,2017,inscri,-1,-1
1,0671864M,N,10,1993,1,S,2014,D,-1,D,...,0,4,M,-1,0,UNIVERSITE STRASBOURG,2017,inscri,-1,-1
2,0673021V,N,10,1993,1,S,2014,D,-1,D,...,0,4,X,-1,0,UNIVERSITE STRASBOURG,2017,inscri,-1,-1
3,0671864M,O,10,1997,1,S,2015,D,-1,A,...,1,4,L,0,0,UNIVERSITE STRASBOURG,2017,inscri,-1,-1
4,0671864M,N,10,1997,1,S,2015,D,-1,A,...,0,4,M,-1,0,UNIVERSITE STRASBOURG,2017,inscri,-1,-1


In [26]:
[conf.get('var_sise') for conf in CONF]

['acabac',
 'age',
 'amena',
 'anbac',
 'annais',
 'bac',
 'bac_rgrp',
 'bac_spe1',
 'bac_spe2',
 'cometa',
 'comins',
 'compos',
 'conv',
 'curpar',
 'cursus_lmd',
 'cycle',
 'degetu',
 'depbac',
 'deprespa',
 'dipder',
 'diplom',
 'echang',
 'effectif',
 'etabli',
 'etabli_diffusion',
 'exoins',
 'flag_meef',
 'flag_sup',
 'groupe',
 'inspr',
 'nation',
 'nature',
 'nbach',
 'net',
 'niveau',
 'niveau_d',
 'niveau_f',
 'numed',
 'oppos',
 'paripa',
 'par_type',
 'pcspar2',
 'pcspar',
 'regime',
 'sectdis',
 'sexe',
 'situpre',
 'specia',
 'specib',
 'specic',
 'typ_dipl',
 'typrepa',
 'voie']

In [3]:
from utils.vars_in_nomen import vbcn
from utils.bcn_load import bcn_complete
bcn=bcn_complete()

Le fichier zip le plus récent est : C:/Users/zfriant/OneDrive/Inscrits/bcn/BCN_EXTRACT_530_5_20250720_083143.zip
Liste des DataFrames :
N_TYPE_DIPLOME_GENERIQUE_SISE
N_TYPE_CLASSE_PREPA_SISE
N_NIVEAU_SISE
N_ORIGINE_RESSOURCE_SISE
N_BCE_SISE_N
N_DOMAINE_FORMATION_SISE
N_ACADEMIE_ET_ASSIMILE
N_PAYS_HISTO
N_SESSION_EXAMEN
N_VOIE_LMD
N_REUSSITE_DIPLOME_SISE
N_TYPE_DIPLOME_SISE
N_CYCLE_SISE
N_COMMUNE
N_DIPLOME_ETAPE_SISE
N_DUREE_ECHANGE_SISE
N_AUTRE_CURSUS_SISE
N_TYPE_ETABLISSEMENT_SISE
N_FORMATIONS_SISE_HISTO
N_GROUPE_DISCIPLINE_SISE
N_SITUATION_ANNEE_PRECEDENTE
N_DERNIER_DIPLOME_OBTENU
N_SECTEUR_DISCIPL_DETAIL_SISE
N_PROGRAMME_ECHANGE_INTERNATIO
N_BAC
N_CURSUS
N_DIPLOME_SISE
N_TYPE_HEBERGEMENT_SISE
N_PRESENCE_EXAMEN
N_AIDE_SISE
N_PARCOURS_TYPE_SISE
N_REUSSITE_EXAMEN_SISE
N_REGIME_INSCRIPTION
N_INSCRIPTION_PRINCIPALE
N_PCS
N_SEXE
N_NATURE_DIPLOME_SISE
N_ACTIVITE_PROFESSIONNELL_SISE
N_ANNEE_DANS_FORMATION_SISE
N_SPECIALISATION
N_BAC_REGROUPE
N_SECTEUR_DISCIPLINAIRE_SISE
N_TELE_ENSEIGNEMENT


In [16]:
work_csv(pd.DataFrame(list(vbcn.items()), columns=['var_sise', 'n_data']), 'sise_config')

In [ ]:
CORRECTIFS_dict = get_all_correctifs()
BCN = bcn_complete()

In [5]:


def vars_sise_to_be_check(year, bcn, com):
    from config_path import PATH
    # from utils.vars_in_nomen import vbcn
    import pandas as pd, json

    CONF=json.load(open('utils/config_sise.json', 'r'))
    vars_sise = pd.read_pickle(f"{PATH}output/items_by_vars{year}.pkl", compression='gzip')

    hors_nomen=pd.DataFrame()

    for conf in CONF:
        var_sise = conf["var_sise"]
        nomen = conf["n_data"]

        if nomen:
            print(f"#### {var_sise}")
            if var_sise in ['cometa', 'comins']:
                l=pd.DataFrame.from_dict(com[nomen]).iloc[:,0].unique()
            else:
                if var_sise in bcn[nomen].columns:
                    l=bcn[nomen][var_sise].unique()
                else:
                    print(f"- le nom de variable {var_sise} n'existe pas dans {nomen}\n - le code suivant va extraire la 1ere colonne {bcn[nomen].columns[0]}")
                    l=bcn[nomen].iloc[:,0].unique()
                
            tmp=vars_sise.loc[(vars_sise.variable==var_sise)].assign(nomenclature=nomen)
            tmp.loc[~tmp.item.isin(l), 'hors_nomenclature'] = '1'
            hors_nomen=pd.concat([hors_nomen, tmp], ignore_index=True)  

        # hors_nomen.to_csv(f"{PATH}test/vars_hs_nomen.csv", sep=';', encoding='utf-8', index=False, na_rep='', float_format='str')
   
    return hors_nomen
vars_sise_to_be_check(2024, BCN, CORRECTIFS_dict)


#### acabac
- le nom de variable acabac n'existe pas dans N_ACADEMIE_ET_ASSIMILE
 - le code suivant va extraire la 1ere colonne academie_et_assimile
#### bac
#### bac_rgrp
- le nom de variable bac_rgrp n'existe pas dans N_BAC_REGROUPE_2
 - le code suivant va extraire la 1ere colonne bac_regroupe_2
#### bac_spe1
- le nom de variable bac_spe1 n'existe pas dans N_SPECIALITE_BAC_2021_SISE
 - le code suivant va extraire la 1ere colonne specialite_bac_2021_sise
#### bac_spe2
- le nom de variable bac_spe2 n'existe pas dans N_SPECIALITE_BAC_2021_SISE
 - le code suivant va extraire la 1ere colonne specialite_bac_2021_sise
#### depbac
- le nom de variable depbac n'existe pas dans N_DEPARTEMENT
 - le code suivant va extraire la 1ere colonne departement_insee_3
#### inspr
- le nom de variable inspr n'existe pas dans N_INSCRIPTION_PRINCIPALE
 - le code suivant va extraire la 1ere colonne inscription_principale
#### cometa
#### comins
#### compos
- le nom de variable compos n'existe pas dans N_BCE_S

KeyError: 'PAYSAGE_id'

In [ ]:

hors_nomen=pd.DataFrame()
for k,v in vbcn.items():
    bcn[v].columns=bcn[v].columns.str.lower()
    if k in bcn[v].columns:
        l=bcn[v][k].unique()
        # tmp=vars_sise.loc[(vars_sise.variable==k)&(~vars_sise.item.isin(l))].value_counts(vs_cols_base, dropna=False).reset_index()
        # hors_nomen=pd.concat([hors_nomen, tmp], ignore_index=True)
    else:
        print(f"- le nom de variable {k} n'existe pas dans {v}\nla 1ere colonne est {bcn[v].iloc[:,0]}, le code suivant va extraire la 1ere colonne")
        l=bcn[v].iloc[:,0].unique()
        tmp=vars_sise.loc[(vars_sise.variable==k)&(~vars_sise.item.isin(l))]
        hors_nomen=pd.concat([hors_nomen, tmp], ignore_index=True)      

hors_nomen=hors_nomen.pivot_table(index=['rentree', 'variable', 'item'], columns='source',   values='count', 
    aggfunc='sum').reset_index()
hors_nomen.fillna('',inplace=True) 

In [ ]:

# work_csv(hors_nomen, 'modalites_hsn')

In [ ]:

test=pd.DataFrame()
for k,v in vbcn.items():
    bcn[v].columns=bcn[v].columns.str.lower()
    l=bcn[v].iloc[:,0].unique()
    tmp=vars_sise.loc[(vars_sise.variable==k)&(~vars_sise.item.isin(l))].value_counts(vs_cols_base, dropna=False).reset_index()
    # hors_nomen=pd.concat([hors_nomen, tmp], ignore_index=T:rue)   

In [ ]:
# for k,v in vbcn.items():
#     bcn[v].columns=bcn[v].columns.str.lower()
#     l=bcn[v].iloc[:,0].unique()
#     print(f"{v} -> {l}")

In [ ]:
for i in bcn:
    x=bcn[i]
    print(f"{i} -> {x.iloc[:,0].unique()}")


In [26]:
bcn['N_SPECIALISATION'][bcn['N_SPECIALISATION'].specialisation=='0000']

,specialisation,libelle_1_specialisation,libelle_2_specialisation,date_ouverture,date_fermeture,date_intervention,gestion_diffusion,n_commentaire,variable_name


In [ ]:
work_csv(bcn['N_PARCOURS_TYPE_SISE'], 'parcours')

In [58]:
t=pd.read_csv(f"{PATH}sise_config.csv", encoding='utf-8', na_values=' ', keep_default_na=False, sep=';', dtype='str')
t.to_json('utils/config_sise.json', orient='records', compression='infer', indent=4)

In [63]:
from utils.config import sise_config
CONF=sise_config()

In [65]:
import json
json.load(open('utils/config_sise.json', 'r'))

[{'var_sise': 'acabac',
  'n_data': 'N_ACADEMIE_ET_ASSIMILE',
  'format': 'str',
  'missing_value': '99'},
 {'var_sise': 'age', 'n_data': '', 'format': 'str', 'missing_value': '-1'},
 {'var_sise': 'amena',
  'n_data': 'N_AMENA',
  'format': 'str',
  'missing_value': ''},
 {'var_sise': 'anbac', 'n_data': '', 'format': 'str', 'missing_value': ''},
 {'var_sise': 'annais', 'n_data': '', 'format': 'str', 'missing_value': '-1'},
 {'var_sise': 'bac', 'n_data': 'N_BAC', 'format': 'str', 'missing_value': ''},
 {'var_sise': 'bac_rgrp',
  'n_data': 'N_BAC_REGROUPE_2',
  'format': 'str',
  'missing_value': ''},
 {'var_sise': 'bac_spe1',
  'n_data': 'N_SPECIALITE_BAC_2021_SISE',
  'format': 'str',
  'missing_value': ''},
 {'var_sise': 'bac_spe2',
  'n_data': 'N_SPECIALITE_BAC_2021_SISE',
  'format': 'str',
  'missing_value': ''},
 {'var_sise': 'cometa',
  'n_data': 'LES_COMMUNES',
  'format': 'str',
  'missing_value': ''},
 {'var_sise': 'comins',
  'n_data': 'LES_COMMUNES',
  'format': 'str',
  'mi